### Limpieza de los accidentes en México

In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
# juntamos todos los accidentes en uno solo archivo.
# get data file names
path =r'../data/atus_anual_1997_2019/conjunto_de_datos/'
filenames = glob.glob(path + "/*.csv")
dfs = []
# datos que nos interesan. 
col_analisis=['id_entidad', 'id_municipio', 'anio', 'mes', 'id_hora',
       'id_minuto', 'id_dia', 'diasemana', 'tipaccid',
        'causaacci', 'caparod', 'sexo', 'aliento',
       'cinturon', 'id_edad', "clasacc"]

for filename in filenames:
    df_year=pd.read_csv(filename, index_col=False)
    df_year.columns = [item.lower() for item in df_year.columns.to_list()]
    dfs.append(df_year[col_analisis])
    print(filename)


# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)
big_frame.to_csv("../data/atus_anual_1997_2019.csv", index=False)

../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2017.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2019.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2003.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2012.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2001.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2010.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2004.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2005.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_1999.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2002.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2018.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2013.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2016.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2008.csv
../data/atus_anual_1997_2019/conjunto_de_datos/atus_anual_2009

In [2]:
accidentes_mexico = pd.read_csv("../data/raw/atus_anual_1997_2019.csv")

In [3]:
accidentes_mexico.shape


(8876380, 16)

In [4]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc
0,1,1.0,2017,1,18,30,11,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30,Sólo daños
1,1,1.0,2017,1,1,50,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30,No fatal
2,1,1.0,2017,1,8,30,12,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22,No fatal
3,1,1.0,2017,1,17,40,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73,Sólo daños
4,1,1.0,2017,1,21,10,11,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33,No fatal


In [5]:
accidentes_mexico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8876380 entries, 0 to 8876379
Data columns (total 16 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id_entidad    int64  
 1   id_municipio  float64
 2   anio          int64  
 3   mes           int64  
 4   id_hora       int64  
 5   id_minuto     int64  
 6   id_dia        int64  
 7   diasemana     object 
 8   tipaccid      object 
 9   causaacci     object 
 10  caparod       object 
 11  sexo          object 
 12  aliento       object 
 13  cinturon      object 
 14  id_edad       int64  
 15  clasacc       object 
dtypes: float64(1), int64(7), object(8)
memory usage: 1.1+ GB


In [6]:
print(accidentes_mexico.id_entidad.unique())

# concatenamos con los catalogos
cat_estado=pd.read_csv("../data/raw/atus_anual_1997_2019/catalogos/tc_entidad.csv", index_col=False)
cat_estado.columns = [item.lower() for item in cat_estado.columns.to_list()]

print(accidentes_mexico.shape)
accidentes_mexico=pd.merge(accidentes_mexico, cat_estado, on="id_entidad", how="left")
print(accidentes_mexico.shape)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
(8876380, 16)
(8876380, 17)


In [7]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad
0,1,1.0,2017,1,18,30,11,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30,Sólo daños,Aguascalientes
1,1,1.0,2017,1,1,50,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30,No fatal,Aguascalientes
2,1,1.0,2017,1,8,30,12,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22,No fatal,Aguascalientes
3,1,1.0,2017,1,17,40,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73,Sólo daños,Aguascalientes
4,1,1.0,2017,1,21,10,11,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33,No fatal,Aguascalientes


In [8]:
print(len(accidentes_mexico.id_municipio.unique()))

# concatenamos con los catalogos
cat_muni=pd.read_csv("../data/raw/atus_anual_1997_2019/catalogos/tc_municipio.csv", index_col=False)
cat_muni.columns = [item.lower() for item in cat_muni.columns.to_list()]

print(accidentes_mexico.shape)
accidentes_mexico=pd.merge(accidentes_mexico, cat_muni, on=["id_entidad","id_municipio"], how="left")
print(accidentes_mexico.shape)

571
(8876380, 17)
(8876380, 18)


In [9]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
0,1,1.0,2017,1,18,30,11,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30,Sólo daños,Aguascalientes,Aguascalientes
1,1,1.0,2017,1,1,50,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30,No fatal,Aguascalientes,Aguascalientes
2,1,1.0,2017,1,8,30,12,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22,No fatal,Aguascalientes,Aguascalientes
3,1,1.0,2017,1,17,40,9,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73,Sólo daños,Aguascalientes,Aguascalientes
4,1,1.0,2017,1,21,10,11,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33,No fatal,Aguascalientes,Aguascalientes


In [10]:
# imprimimos los años
accidentes_mexico.anio.unique()

array([2017, 2019, 2003, 2012, 2001, 2010, 2004, 2005, 1999, 2002, 2018,
       2013, 2016, 2008, 2009, 2014, 2000, 2007, 2011, 1998, 2015, 1997,
       2006])

In [11]:
accidentes_mexico.mes.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [12]:
accidentes_mexico[accidentes_mexico.id_dia==0].head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
3934119,1,8.0,2018,1,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,San José de Gracia
3934425,1,8.0,2018,2,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,San José de Gracia
3935055,1,4.0,2018,4,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,Cosío
3935120,1,10.0,2018,4,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,El Llano
3935376,1,4.0,2018,5,0,0,0,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,Certificado cero,0,Certificado cero,Aguascalientes,Cosío


In [13]:
# analicemos la existencia de ceros en los días
accidentes_dia_cero = accidentes_mexico[accidentes_mexico.id_dia==0]

In [14]:
accidentes_dia_cero.anio.unique() # todos estan en el año 2018

array([2018])

In [15]:
accidentes_dia_cero.nom_entidad.value_counts() # la mayoria estan en Oaxaca.

Oaxaca                             6361
Puebla                             2207
Veracruz de Ignacio de la Llave    1584
Chiapas                            1128
Hidalgo                             865
San Luis Potosi                     434
Sonora                              431
Tlaxcala                            348
Tamaulipas                          265
Michoacan de Ocampo                 251
Yucatan                             227
Mexico                              211
Nuevo Leon                          164
Zacatecas                           164
Chihuahua                           150
Coahuila de Zaragoza                137
Nayarit                              73
Durango                              59
Guanajuato                           55
Tabasco                              25
Queretaro                            21
Aguascalientes                       19
Morelos                              11
Campeche                             11
Sinaloa                               7


In [16]:
for i in accidentes_dia_cero.columns[4:16]:
    print("La columna",i,"tiene los valores únicos:", accidentes_dia_cero[i].unique())

La columna id_hora tiene los valores únicos: [0]
La columna id_minuto tiene los valores únicos: [0]
La columna id_dia tiene los valores únicos: [0]
La columna diasemana tiene los valores únicos: ['Certificado cero']
La columna tipaccid tiene los valores únicos: ['Certificado cero']
La columna causaacci tiene los valores únicos: ['Certificado cero']
La columna caparod tiene los valores únicos: ['Certificado cero']
La columna sexo tiene los valores únicos: ['Certificado cero']
La columna aliento tiene los valores únicos: ['Certificado cero']
La columna cinturon tiene los valores únicos: ['Certificado cero']
La columna id_edad tiene los valores únicos: [0]
La columna clasacc tiene los valores únicos: ['Certificado cero']


In [17]:
# Filtramos los que tenga en día 0.
accidentes_mexico = accidentes_mexico[accidentes_mexico.id_dia!=0]

In [18]:
accidentes_mexico.id_dia.unique()

array([11,  9, 12, 13, 10, 14, 29, 15, 30, 31, 16,  7,  5,  3,  1, 17, 18,
       22, 27, 28, 26,  2, 19, 25,  6,  4, 21, 20,  8, 23, 24, 32])

In [27]:
accidentes_mexico["id_dia"] = accidentes_mexico["id_dia"].replace({np.nan:1})

In [20]:
accidentes_mexico.diasemana.unique()

array(['Miércoles', 'Lunes', 'Jueves', 'Viernes', 'Martes', 'Sábado',
       'Domingo', 'Miercoles', 'Sabado', 'lunes', 'No especificado'],
      dtype=object)

In [21]:
accidentes_mexico["id_edad"]=accidentes_mexico["id_edad"].replace({0:np.nan,
                                                                  99:np.nan})

In [22]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
0,1,1.0,2017,1,18,30,11.0,Miércoles,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,30.0,Sólo daños,Aguascalientes,Aguascalientes
1,1,1.0,2017,1,1,50,9.0,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Hombre,No,Se ignora,30.0,No fatal,Aguascalientes,Aguascalientes
2,1,1.0,2017,1,8,30,12.0,Jueves,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,22.0,No fatal,Aguascalientes,Aguascalientes
3,1,1.0,2017,1,17,40,9.0,Lunes,Colisión con motocicleta,Conductor,Pavimentada,Mujer,No,Se ignora,73.0,Sólo daños,Aguascalientes,Aguascalientes
4,1,1.0,2017,1,21,10,11.0,Miércoles,Colisión con objeto fijo,Conductor,Pavimentada,Hombre,No,Se ignora,33.0,No fatal,Aguascalientes,Aguascalientes


In [28]:
for i in ["anio", "mes"]:
    accidentes_mexico[i]=accidentes_mexico[i].astype("str")

accidentes_mexico["id_dia"]=accidentes_mexico["id_dia"].astype(int).astype("str")

In [29]:
accidentes_mexico["id_dia"].unique()

array(['11', '9', '12', '13', '10', '14', '29', '15', '30', '31', '16',
       '7', '5', '3', '1', '17', '18', '22', '27', '28', '26', '2', '19',
       '25', '6', '4', '21', '20', '8', '23', '24'], dtype=object)

In [ ]:
accidentes_mexico["fecha"] = pd.to_datetime(accidentes_mexico["id_dia"]+"-"+accidentes_mexico["mes"]+"-"+
                                           accidentes_mexico["anio"])

In [32]:
accidentes_mexico.to_csv("../data/processed/atus_anual_1997_2019.csv", index=False)

## Limpieza de los vehiculos registrados en circulación

In [1]:
import pandas as pd
import glob
import numpy as np

In [13]:
vehiculos = pd.read_csv("../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_1997.csv", index_col=False)
vehiculos.columns = [item.lower() for item in vehiculos.columns.to_list()]
vehiculos.head(3)

,prod_est,cobertura,anio,id_entidad,id_municipio,auto_oficial,auto_publico,auto_particular,cam_pas_oficial,cam_pas_publico,cam_pas_particular,cyc_carga_oficial,cyc_carga_publico,cyc_carga_particular,moto_oficial,moto_de_alquiler,moto_particular,estatus
0,Vehículos de motor registrados en circulación....,Municipal,1997,1,1,1233,2212,81265,20,725,316,906,277,37905,146,0,6231,Cifras Definitivas.
1,Vehículos de motor registrados en circulación....,Municipal,1997,1,2,3,8,529,0,13,1,6,0,1589,0,0,13,Cifras Definitivas.
2,Vehículos de motor registrados en circulación....,Municipal,1997,1,3,9,63,2155,0,40,6,20,0,5124,2,0,69,Cifras Definitivas.


In [14]:
vehiculos.shape

(1495, 18)

In [15]:
vehiculos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   prod_est              1495 non-null   object
 1   cobertura             1495 non-null   object
 2   anio                  1495 non-null   int64 
 3   id_entidad            1495 non-null   int64 
 4   id_municipio          1495 non-null   int64 
 5   auto_oficial          1495 non-null   int64 
 6   auto_publico          1495 non-null   int64 
 7   auto_particular       1495 non-null   int64 
 8   cam_pas_oficial       1495 non-null   int64 
 9   cam_pas_publico       1495 non-null   int64 
 10  cam_pas_particular    1495 non-null   int64 
 11  cyc_carga_oficial     1495 non-null   int64 
 12  cyc_carga_publico     1495 non-null   int64 
 13  cyc_carga_particular  1495 non-null   int64 
 14  moto_oficial          1495 non-null   int64 
 15  moto_de_alquiler      1495 non-null   

In [19]:
for i in ['prod_est', 'cobertura', 'anio', 'id_entidad', 'id_municipio']:
    print("Columna:" , i, "valores unicos", vehiculos[i].unique())

Columna: prod_est valores unicos ['Vehículos de motor registrados en circulación. Anual']
Columna: cobertura valores unicos ['Municipal']
Columna: anio valores unicos [1997]
Columna: id_entidad valores unicos [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
Columna: id_municipio valores unicos [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  40  57  59  61  65  68  74  78  81  89  97 101 106 108 109  39
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  58  60
  62  63  64  66  67  73  75  69  70  71  72  76  77  79  80  82  83  84
  85  86  87  88  90  91  92  93  94  95  96  98  99 100 102 103 104 105
 107 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124   0 132
 140 153 154 156 164 169 172 174 186 197 207 208 125 126 127 128 129 130
 131 133 134 135 136 137 138 139 141 142 143 144 145 146 147 148 149 15

In [31]:
vehiculos = vehiculos.groupby(["anio", "id_entidad","id_municipio"], as_index=False)[vehiculos.columns[5:]].sum()

Por lo tanto, solo se necesitaran las columnas: "anio", "id_entidad", "id_municipio", 'auto_oficial', 'auto_publico', 'auto_particular', 'cam_pas_oficial', 'cam_pas_publico', 'cam_pas_particular', 'cyc_carga_oficial', 'cyc_carga_publico', 'cyc_carga_particular', 'moto_oficial','moto_de_alquiler', 'moto_particular'.

In [37]:
# juntamos todos los accidentes en uno solo archivo.
# get data file names
path =r'../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/'
filenames = glob.glob(path + "/*.csv")
dfs = []
# datos que nos interesan. 
col_analisis=["anio", "id_entidad", "id_municipio", 'auto_oficial', 'auto_publico', 'auto_particular', 
              'cam_pas_oficial', 'cam_pas_publico', 'cam_pas_particular', 'cyc_carga_oficial', 
              'cyc_carga_publico', 'cyc_carga_particular', 'moto_oficial','moto_de_alquiler', 
              'moto_particular']

for filename in filenames:
    df_year=pd.read_csv(filename, index_col=False)
    df_year.columns = [item.lower() for item in df_year.columns.to_list()]
    dfs.append(df_year[col_analisis])
    print(filename)

# Concatenate all data into one DataFrame
vehiculos_datos_completos = pd.concat(dfs, ignore_index=True)

../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2019.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2012.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2001.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2007.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2009.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2006.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_1999.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2010.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2013.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2017.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2018.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2004.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_1998.csv
../data/raw/vmrc_anual_csv_1980_2019/conjunto_de_datos/vmrc_2000.csv
../data/raw/vmrc_anual_csv_1980_20

In [39]:
for i in ['anio', 'id_entidad', 'id_municipio']:
    print("Columna:" , i, "valores unicos", vehiculos_datos_completos[i].unique())

Columna: anio valores unicos [2019 2012 2001 2007 2009 2006 1999 2010 2013 2017 2018 2004 1998 2000
 2014 2005 2015 1997 2016 2003 2008 2002 2011]
Columna: id_entidad valores unicos [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
Columna: id_municipio valores unicos [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 122 123 124 125  95 120 121 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 

In [45]:
vehiculos_datos_completos.head()

,anio,id_entidad,id_municipio,auto_oficial,auto_publico,auto_particular,cam_pas_oficial,cam_pas_publico,cam_pas_particular,cyc_carga_oficial,cyc_carga_publico,cyc_carga_particular,moto_oficial,moto_de_alquiler,moto_particular
0,2019,1,1,466,4251,307832,9,2478,730,633,3545,99824,234,1,54280
1,2019,1,2,1,36,5504,0,5,18,3,21,5068,4,0,3681
2,2019,1,3,2,157,11110,0,9,11,16,21,12212,10,0,1711
3,2019,1,4,1,16,2279,0,0,4,6,0,1708,0,0,1031
4,2019,1,5,22,205,32039,0,80,96,38,192,14334,23,0,6672


In [47]:
vehiculos_datos_completos[vehiculos_datos_completos.id_municipio.isin([0,999])]

,anio,id_entidad,id_municipio,auto_oficial,auto_publico,auto_particular,cam_pas_oficial,cam_pas_publico,cam_pas_particular,cyc_carga_oficial,cyc_carga_publico,cyc_carga_particular,moto_oficial,moto_de_alquiler,moto_particular
2645,2012,9,999,0,14445,653921,0,0,0,0,0,0,0,0,33
5097,2001,13,999,0,0,81,0,0,7,0,2,310,0,0,0
5559,2001,20,0,1126,5796,104911,83,1847,446,3822,2706,91401,249,120,7721
6501,2007,9,999,0,4548,268609,0,0,4841,0,122,3890,0,0,0
7166,2007,20,0,1488,9346,137153,136,1070,503,3621,4768,124070,402,134,11705
8311,2009,9,999,0,25322,666871,0,0,8421,0,196,6237,0,0,0
10659,2006,9,999,0,4572,274477,0,0,4834,0,104,3313,0,0,0
11319,2006,20,0,1413,7820,123783,129,928,425,3342,4078,108791,367,130,9582
13027,1999,20,0,968,2751,90135,52,1734,328,2316,1452,70369,127,0,4166
13974,2010,9,999,0,18917,660967,0,0,8336,0,215,6780,0,0,0


Los estados 9, 13, 20 y 7 presentan valores nulos. 

In [48]:
# concatenamos con los catalogos
cat_estado=pd.read_csv("../data/raw/vmrc_anual_csv_1980_2019/catalogos/tc_entidad.csv", index_col=False)
cat_estado.columns = [item.lower() for item in cat_estado.columns.to_list()]

print(vehiculos_datos_completos.shape)
vehiculos_datos_completos=pd.merge(vehiculos_datos_completos, cat_estado, on="id_entidad", how="left")
print(vehiculos_datos_completos.shape)

(46300, 15)
(46300, 16)


In [50]:
cat_municipio = pd.read_csv("../data/raw/vmrc_anual_csv_1980_2019/catalogos/tc_municipio.csv", index_col=False)
cat_municipio.columns = [item.lower() for item in cat_municipio.columns.to_list()]

print(vehiculos_datos_completos.shape)
vehiculos_datos_completos=pd.merge(vehiculos_datos_completos, cat_municipio, on=["id_entidad", "id_municipio"],
                                   how="left")
print(vehiculos_datos_completos.shape)

(46300, 16)
(46300, 17)


In [51]:
vehiculos_datos_completos.head()

,anio,id_entidad,id_municipio,auto_oficial,auto_publico,auto_particular,cam_pas_oficial,cam_pas_publico,cam_pas_particular,cyc_carga_oficial,cyc_carga_publico,cyc_carga_particular,moto_oficial,moto_de_alquiler,moto_particular,nom_entidad,nom_municipio
0,2019,1,1,466,4251,307832,9,2478,730,633,3545,99824,234,1,54280,Aguascalientes,Aguascalientes
1,2019,1,2,1,36,5504,0,5,18,3,21,5068,4,0,3681,Aguascalientes,Asientos
2,2019,1,3,2,157,11110,0,9,11,16,21,12212,10,0,1711,Aguascalientes,Calvillo
3,2019,1,4,1,16,2279,0,0,4,6,0,1708,0,0,1031,Aguascalientes,Cosío
4,2019,1,5,22,205,32039,0,80,96,38,192,14334,23,0,6672,Aguascalientes,Jesús María


In [52]:
vehiculos_datos_completos.to_csv("../data/processed/vehiculos_anual_1997_2019.csv", index=False)

## Limpieza datos vehiculos mensuales

In [40]:
vehiculos_mensual = pd.read_csv("../data/raw/vmcr_vehiculos_mensual_1991_2018/conjunto_de_datos/vmrc_vehiculos_mensual_tr_cifra_1991_2018.csv", 
                                index_col=False)

In [41]:
vehiculos_mensual.head()

,PROD_EST,COBERTURA,ANIO,ID_MES,AUTO_PUBLICO,AUTO_PARTICULAR,CAM_PAS_PUBLICO,CAM_PAS_PARTICULAR,CYC_CARGA_PUBLICO,CYC_CARGA_PARTICULAR,ESTATUS
0,Vehículos de motor registrados en circulación....,Nacional,1991,1,258855,6296802,78900,11797,106727,2867778,Cifras Definitivas.
1,Vehículos de motor registrados en circulación....,Nacional,1991,2,260130,6327813,79141,11833,107673,2893193,Cifras Definitivas.
2,Vehículos de motor registrados en circulación....,Nacional,1991,3,261408,6358897,79382,11869,108625,2918787,Cifras Definitivas.
3,Vehículos de motor registrados en circulación....,Nacional,1991,4,262689,6390062,79624,11906,109585,2944568,Cifras Definitivas.
4,Vehículos de motor registrados en circulación....,Nacional,1991,5,263974,6421311,79868,11942,110552,2970546,Cifras Definitivas.


Existen datos de los vehiculos registrados por mes desde 1991 a 2018, pero no se encuentran a nivel municipal por lo que se considera que estos datos no son servirián para el analisis buscado. 

In [42]:
vehiculos_mensual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   PROD_EST              356 non-null    object
 1   COBERTURA             356 non-null    object
 2   ANIO                  356 non-null    int64 
 3   ID_MES                356 non-null    int64 
 4   AUTO_PUBLICO          356 non-null    int64 
 5   AUTO_PARTICULAR       356 non-null    int64 
 6   CAM_PAS_PUBLICO       356 non-null    int64 
 7   CAM_PAS_PARTICULAR    356 non-null    int64 
 8   CYC_CARGA_PUBLICO     356 non-null    int64 
 9   CYC_CARGA_PARTICULAR  356 non-null    int64 
 10  ESTATUS               356 non-null    object
dtypes: int64(8), object(3)
memory usage: 30.7+ KB
